In [1]:
# import xgboost as xgb
# from sklearn.model_selection import StratifiedKFold
# from sklearn.metrics import cohen_kappa_score
# import numpy as np
# import pandas as pd
# from tqdm import tqdm
# from sklearn.base import clone
# from concurrent.futures import ThreadPoolExecutor
# import os
# from scipy.optimize import minimize
# from colorama import Fore, Style

# # Load data
# train = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/train.csv')
# test = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/test.csv')
# sample = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/sample_submission.csv')

# # Function to process time series data
# def process_file(filename, dirname):
#     df = pd.read_parquet(os.path.join(dirname, filename, 'part-0.parquet'))
#     df.drop('step', axis=1, inplace=True)
#     return df.describe().values.reshape(-1), filename.split('=')[1]

# def load_time_series(dirname) -> pd.DataFrame:
#     ids = os.listdir(dirname)
    
#     with ThreadPoolExecutor() as executor:
#         results = list(tqdm(executor.map(lambda fname: process_file(fname, dirname), ids), total=len(ids)))
    
#     stats, indexes = zip(*results)
    
#     df = pd.DataFrame(stats, columns=[f"stat_{i}" for i in range(len(stats[0]))])
#     df['id'] = indexes
#     return df
        
# train_ts = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet")
# test_ts = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet")

# time_series_cols = train_ts.columns.tolist()
# time_series_cols.remove("id")

# train = pd.merge(train, train_ts, how="left", on='id')
# test = pd.merge(test, test_ts, how="left", on='id')

# train = train.drop('id', axis=1)
# test = test.drop('id', axis=1)   

# # Feature columns
# featuresCols = ['Basic_Demos-Enroll_Season', 'Basic_Demos-Age', 'Basic_Demos-Sex',
#                 'CGAS-Season', 'CGAS-CGAS_Score', 'Physical-Season', 'Physical-BMI',
#                 'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
#                 'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
#                 'Fitness_Endurance-Season', 'Fitness_Endurance-Max_Stage',
#                 'Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec',
#                 'FGC-Season', 'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND',
#                 'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone', 'FGC-FGC_PU',
#                 'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR',
#                 'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone', 'BIA-Season',
#                 'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
#                 'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
#                 'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
#                 'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM',
#                 'BIA-BIA_TBW', 'PAQ_A-Season', 'PAQ_A-PAQ_A_Total', 'PAQ_C-Season',
#                 'PAQ_C-PAQ_C_Total', 'SDS-Season', 'SDS-SDS_Total_Raw',
#                 'SDS-SDS_Total_T', 'PreInt_EduHx-Season',
#                 'PreInt_EduHx-computerinternet_hoursday', 'sii']

# featuresCols += time_series_cols

# train = train[featuresCols]
# train = train.dropna(subset='sii')

# cat_c = ['Basic_Demos-Enroll_Season', 'CGAS-Season', 'Physical-Season', 
#           'Fitness_Endurance-Season', 'FGC-Season', 'BIA-Season', 
#           'PAQ_A-Season', 'PAQ_C-Season', 'SDS-Season', 'PreInt_EduHx-Season']

# def update(df):
#     global cat_c
#     for c in cat_c: 
#         df[c] = df[c].fillna('Missing')
#         df[c] = df[c].astype('category')
#     return df
        
# train = update(train)
# test = update(test)

# def create_mapping(column, dataset):
#     unique_values = dataset[column].unique()
#     return {value: idx for idx, value in enumerate(unique_values)}

# for col in cat_c:
#     mapping = create_mapping(col, train)
#     mappingTe = create_mapping(col, test)
    
#     train[col] = train[col].replace(mapping).astype(int)
#     test[col] = test[col].replace(mappingTe).astype(int)

# def quadratic_weighted_kappa(y_true, y_pred):
#     return cohen_kappa_score(y_true, y_pred, weights='quadratic')
# def threshold_Rounder(oof_non_rounded, thresholds):
#     return np.where(oof_non_rounded < thresholds[0], 0,
#                     np.where(oof_non_rounded < thresholds[1], 1,
#                              np.where(oof_non_rounded < thresholds[2], 2, 3)))
# def evaluate_predictions(thresholds, y_true, oof_non_rounded):
#     rounded_p = threshold_Rounder(oof_non_rounded, thresholds)
#     return -quadratic_weighted_kappa(y_true, rounded_p)


# def TrainML(model_class, test_data):
#     X = train.drop(['sii'], axis=1)
#     y = train['sii']

#     SKF = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
#     train_S = []
#     test_S = []
    
#     oof_non_rounded = np.zeros(len(y), dtype=float) 
#     oof_rounded = np.zeros(len(y), dtype=int) 
#     test_preds = np.zeros((len(test_data), 5))

#     for fold, (train_idx, test_idx) in enumerate(tqdm(SKF.split(X, y), desc="Training Folds", total=5)):
#         X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
#         y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]

#         model = clone(model_class)
#         model.fit(X_train, y_train)

#         y_train_pred = model.predict(X_train)
#         y_val_pred = model.predict(X_val)

#         oof_non_rounded[test_idx] = y_val_pred
#         y_val_pred_rounded = y_val_pred.round(0).astype(int)
#         oof_rounded[test_idx] = y_val_pred_rounded

#         train_kappa = quadratic_weighted_kappa(y_train, y_train_pred.round(0).astype(int))
#         val_kappa = quadratic_weighted_kappa(y_val, y_val_pred_rounded)

#         train_S.append(train_kappa)
#         test_S.append(val_kappa)
        
#         test_preds[:, fold] = model.predict(test_data)
        
#         print(f"Fold {fold+1} - Train QWK: {train_kappa:.4f}, Validation QWK: {val_kappa:.4f}")
    
#     print(f"Mean Train QWK --> {np.mean(train_S):.4f}")
#     print(f"Mean Validation QWK ---> {np.mean(test_S):.4f}")

#     KappaOPtimizer = minimize(evaluate_predictions,
#                               x0=[0.5, 1.5, 2.5], args=(y, oof_non_rounded), 
#                               method='Nelder-Mead')
#     assert KappaOPtimizer.success, "Optimization did not converge."
    
#     oof_tuned = threshold_Rounder(oof_non_rounded, KappaOPtimizer.x)
#     tKappa = quadratic_weighted_kappa(y, oof_tuned)

#     print(f"----> || Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {tKappa:.3f}{Style.RESET_ALL}")
    
#     tpm = test_preds.mean(axis=1)
#     tpTuned = threshold_Rounder(tpm, KappaOPtimizer.x)
    

#     # Create final submission prediction
#     submission = pd.DataFrame({
#         'id': sample['id'],
#         'sii': test_preds.mean(axis=1).round(0).astype(int)
#     })

#     return submission

# # XGBoost parameters
# Params = {
#     'learning_rate': 0.05,
#     'max_depth': 6,
#     'n_estimators': 200,
#     'subsample': 0.8,
#     'colsample_bytree': 0.8,
#     'reg_alpha': 1,  # Increased from 0.1
#     'reg_lambda': 5,  # Increased from 1
#     'random_state': 42
# }


# # Instantiate the XGBoost model
# XGBoost = xgb.XGBRegressor(**Params, verbosity=0)

# # Train the model and get predictions
# Submission = TrainML(XGBoost, test)

# # Output submission
# Submission.to_csv('submission.csv', index=False)


In [2]:
# from catboost import CatBoostRegressor
# from sklearn.model_selection import StratifiedKFold
# from sklearn.metrics import cohen_kappa_score
# import numpy as np
# import pandas as pd
# from tqdm import tqdm
# from sklearn.base import clone
# from scipy.optimize import minimize
# from concurrent.futures import ThreadPoolExecutor
# import os
# from colorama import Fore, Style

# # Load data
# train = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/train.csv')
# test = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/test.csv')
# sample = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/sample_submission.csv')

# # Function to process time series data
# def process_file(filename, dirname):
#     df = pd.read_parquet(os.path.join(dirname, filename, 'part-0.parquet'))
#     df.drop('step', axis=1, inplace=True)
#     return df.describe().values.reshape(-1), filename.split('=')[1]

# def load_time_series(dirname) -> pd.DataFrame:
#     ids = os.listdir(dirname)
    
#     with ThreadPoolExecutor() as executor:
#         results = list(tqdm(executor.map(lambda fname: process_file(fname, dirname), ids), total=len(ids)))
    
#     stats, indexes = zip(*results)
    
#     df = pd.DataFrame(stats, columns=[f"stat_{i}" for i in range(len(stats[0]))])
#     df['id'] = indexes
#     return df
        
# train_ts = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet")
# test_ts = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet")

# time_series_cols = train_ts.columns.tolist()
# time_series_cols.remove("id")

# train = pd.merge(train, train_ts, how="left", on='id')
# test = pd.merge(test, test_ts, how="left", on='id')

# train = train.drop('id', axis=1)
# test = test.drop('id', axis=1)   

# # Feature columns
# featuresCols = ['Basic_Demos-Enroll_Season', 'Basic_Demos-Age', 'Basic_Demos-Sex',
#                 'CGAS-Season', 'CGAS-CGAS_Score', 'Physical-Season', 'Physical-BMI',
#                 'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
#                 'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
#                 'Fitness_Endurance-Season', 'Fitness_Endurance-Max_Stage',
#                 'Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec',
#                 'FGC-Season', 'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND',
#                 'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone', 'FGC-FGC_PU',
#                 'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR',
#                 'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone', 'BIA-Season',
#                 'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
#                 'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
#                 'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
#                 'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM',
#                 'BIA-BIA_TBW', 'PAQ_A-Season', 'PAQ_A-PAQ_A_Total', 'PAQ_C-Season',
#                 'PAQ_C-PAQ_C_Total', 'SDS-Season', 'SDS-SDS_Total_Raw',
#                 'SDS-SDS_Total_T', 'PreInt_EduHx-Season',
#                 'PreInt_EduHx-computerinternet_hoursday', 'sii']

# featuresCols += time_series_cols

# train = train[featuresCols]
# train = train.dropna(subset='sii')

# cat_c = ['Basic_Demos-Enroll_Season', 'CGAS-Season', 'Physical-Season', 
#           'Fitness_Endurance-Season', 'FGC-Season', 'BIA-Season', 
#           'PAQ_A-Season', 'PAQ_C-Season', 'SDS-Season', 'PreInt_EduHx-Season']

# def update(df):
#     global cat_c
#     for c in cat_c: 
#         df[c] = df[c].fillna('Missing')
#         df[c] = df[c].astype('category')
#     return df
        
# train = update(train)
# test = update(test)

# def create_mapping(column, dataset):
#     unique_values = dataset[column].unique()
#     return {value: idx for idx, value in enumerate(unique_values)}

# for col in cat_c:
#     mapping = create_mapping(col, train)
#     mappingTe = create_mapping(col, test)
    
#     train[col] = train[col].replace(mapping).astype(int)
#     test[col] = test[col].replace(mappingTe).astype(int)

# def quadratic_weighted_kappa(y_true, y_pred):
#     return cohen_kappa_score(y_true, y_pred, weights='quadratic')

# def threshold_Rounder(oof_non_rounded, thresholds):
#     return np.where(oof_non_rounded < thresholds[0], 0,
#                     np.where(oof_non_rounded < thresholds[1], 1,
#                              np.where(oof_non_rounded < thresholds[2], 2, 3)))
# def evaluate_predictions(thresholds, y_true, oof_non_rounded):
#     rounded_p = threshold_Rounder(oof_non_rounded, thresholds)
#     return -quadratic_weighted_kappa(y_true, rounded_p)

# def TrainML(model_class, test_data):
#     X = train.drop(['sii'], axis=1)
#     y = train['sii']

#     SKF = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
#     train_S = []
#     test_S = []
    
#     oof_non_rounded = np.zeros(len(y), dtype=float) 
#     oof_rounded = np.zeros(len(y), dtype=int) 
#     test_preds = np.zeros((len(test_data), 5))

#     for fold, (train_idx, test_idx) in enumerate(tqdm(SKF.split(X, y), desc="Training Folds", total=5)):
#         X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
#         y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]

#         model = clone(model_class)
#         model.fit(X_train, y_train)

#         y_train_pred = model.predict(X_train)
#         y_val_pred = model.predict(X_val)

#         oof_non_rounded[test_idx] = y_val_pred
#         y_val_pred_rounded = y_val_pred.round(0).astype(int)
#         oof_rounded[test_idx] = y_val_pred_rounded

#         train_kappa = quadratic_weighted_kappa(y_train, y_train_pred.round(0).astype(int))
#         val_kappa = quadratic_weighted_kappa(y_val, y_val_pred_rounded)

#         train_S.append(train_kappa)
#         test_S.append(val_kappa)
        
#         test_preds[:, fold] = model.predict(test_data)
        
#         print(f"Fold {fold+1} - Train QWK: {train_kappa:.4f}, Validation QWK: {val_kappa:.4f}")
    
#     print(f"Mean Train QWK --> {np.mean(train_S):.4f}")
#     print(f"Mean Validation QWK ---> {np.mean(test_S):.4f}")

#     KappaOPtimizer = minimize(evaluate_predictions,
#                               x0=[0.5, 1.5, 2.5], args=(y, oof_non_rounded), 
#                               method='Nelder-Mead')
#     assert KappaOPtimizer.success, "Optimization did not converge."
    
#     oof_tuned = threshold_Rounder(oof_non_rounded, KappaOPtimizer.x)
#     tKappa = quadratic_weighted_kappa(y, oof_tuned)

#     print(f"----> || Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {tKappa:.3f}{Style.RESET_ALL}")
    
#     tpm = test_preds.mean(axis=1)
#     tpTuned = threshold_Rounder(tpm, KappaOPtimizer.x)
    

#     # Create final submission prediction
#     submission = pd.DataFrame({
#         'id': sample['id'],
#         'sii': test_preds.mean(axis=1).round(0).astype(int)
#     })

#     return submission

# # CatBoost parameters
# CatBoost_Params = {
#     'learning_rate': 0.05,
#     'depth': 6,
#     'iterations': 200,
#     'random_seed': 42,
#     'verbose': 0,
#     'l2_leaf_reg': 10, 
#     'task_type': 'GPU'
# }

# # Instantiate the CatBoost model
# CatBoost_Model = CatBoostRegressor(**CatBoost_Params)

# # Train the model and get predictions
# Submission = TrainML(CatBoost_Model, test)

# # Output submission
# Submission.to_csv('submission.csv', index=False)


In [3]:
# from lightgbm import LGBMRegressor
# from sklearn.model_selection import StratifiedKFold
# from sklearn.metrics import cohen_kappa_score
# import numpy as np
# import pandas as pd
# from tqdm import tqdm
# from sklearn.base import clone
# from scipy.optimize import minimize
# import os
# from concurrent.futures import ThreadPoolExecutor
# from scipy.optimize import minimize
# from colorama import Fore, Style

# # Load data
# train = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/train.csv')
# test = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/test.csv')
# sample = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/sample_submission.csv')

# # Function to process time series data
# def process_file(filename, dirname):
#     df = pd.read_parquet(os.path.join(dirname, filename, 'part-0.parquet'))
#     df.drop('step', axis=1, inplace=True)
#     return df.describe().values.reshape(-1), filename.split('=')[1]

# def load_time_series(dirname) -> pd.DataFrame:
#     ids = os.listdir(dirname)
    
#     with ThreadPoolExecutor() as executor:
#         results = list(tqdm(executor.map(lambda fname: process_file(fname, dirname), ids), total=len(ids)))
    
#     stats, indexes = zip(*results)
    
#     df = pd.DataFrame(stats, columns=[f"stat_{i}" for i in range(len(stats[0]))])
#     df['id'] = indexes
#     return df
        
# train_ts = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet")
# test_ts = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet")

# time_series_cols = train_ts.columns.tolist()
# time_series_cols.remove("id")

# train = pd.merge(train, train_ts, how="left", on='id')
# test = pd.merge(test, test_ts, how="left", on='id')

# train = train.drop('id', axis=1)
# test = test.drop('id', axis=1)   

# # Feature columns
# featuresCols = ['Basic_Demos-Enroll_Season', 'Basic_Demos-Age', 'Basic_Demos-Sex',
#                 'CGAS-Season', 'CGAS-CGAS_Score', 'Physical-Season', 'Physical-BMI',
#                 'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
#                 'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
#                 'Fitness_Endurance-Season', 'Fitness_Endurance-Max_Stage',
#                 'Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec',
#                 'FGC-Season', 'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND',
#                 'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone', 'FGC-FGC_PU',
#                 'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR',
#                 'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone', 'BIA-Season',
#                 'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
#                 'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
#                 'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
#                 'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM',
#                 'BIA-BIA_TBW', 'PAQ_A-Season', 'PAQ_A-PAQ_A_Total', 'PAQ_C-Season',
#                 'PAQ_C-PAQ_C_Total', 'SDS-Season', 'SDS-SDS_Total_Raw',
#                 'SDS-SDS_Total_T', 'PreInt_EduHx-Season',
#                 'PreInt_EduHx-computerinternet_hoursday', 'sii']

# featuresCols += time_series_cols

# train = train[featuresCols]
# train = train.dropna(subset='sii')

# cat_c = ['Basic_Demos-Enroll_Season', 'CGAS-Season', 'Physical-Season', 
#           'Fitness_Endurance-Season', 'FGC-Season', 'BIA-Season', 
#           'PAQ_A-Season', 'PAQ_C-Season', 'SDS-Season', 'PreInt_EduHx-Season']

# def update(df):
#     global cat_c
#     for c in cat_c: 
#         df[c] = df[c].fillna('Missing')
#         df[c] = df[c].astype('category')
#     return df
        
# train = update(train)
# test = update(test)

# def create_mapping(column, dataset):
#     unique_values = dataset[column].unique()
#     return {value: idx for idx, value in enumerate(unique_values)}

# for col in cat_c:
#     mapping = create_mapping(col, train)
#     mappingTe = create_mapping(col, test)
    
#     train[col] = train[col].replace(mapping).astype(int)
#     test[col] = test[col].replace(mappingTe).astype(int)

# def quadratic_weighted_kappa(y_true, y_pred):
#     return cohen_kappa_score(y_true, y_pred, weights='quadratic')
# def threshold_Rounder(oof_non_rounded, thresholds):
#     return np.where(oof_non_rounded < thresholds[0], 0,
#                     np.where(oof_non_rounded < thresholds[1], 1,
#                              np.where(oof_non_rounded < thresholds[2], 2, 3)))
# def evaluate_predictions(thresholds, y_true, oof_non_rounded):
#     rounded_p = threshold_Rounder(oof_non_rounded, thresholds)
#     return -quadratic_weighted_kappa(y_true, rounded_p)

# def TrainML(model_class, test_data):
#     X = train.drop(['sii'], axis=1)
#     y = train['sii']

#     SKF = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
#     train_S = []
#     test_S = []
    
#     oof_non_rounded = np.zeros(len(y), dtype=float) 
#     oof_rounded = np.zeros(len(y), dtype=int) 
#     test_preds = np.zeros((len(test_data), 5))

#     for fold, (train_idx, test_idx) in enumerate(tqdm(SKF.split(X, y), desc="Training Folds", total=5)):
#         X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
#         y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]

#         model = clone(model_class)
#         model.fit(X_train, y_train)

#         y_train_pred = model.predict(X_train)
#         y_val_pred = model.predict(X_val)

#         oof_non_rounded[test_idx] = y_val_pred
#         y_val_pred_rounded = y_val_pred.round(0).astype(int)
#         oof_rounded[test_idx] = y_val_pred_rounded

#         train_kappa = quadratic_weighted_kappa(y_train, y_train_pred.round(0).astype(int))
#         val_kappa = quadratic_weighted_kappa(y_val, y_val_pred_rounded)

#         train_S.append(train_kappa)
#         test_S.append(val_kappa)
        
#         test_preds[:, fold] = model.predict(test_data)
        
#         print(f"Fold {fold+1} - Train QWK: {train_kappa:.4f}, Validation QWK: {val_kappa:.4f}")
    
#     print(f"Mean Train QWK --> {np.mean(train_S):.4f}")
#     print(f"Mean Validation QWK ---> {np.mean(test_S):.4f}")
#         # After the for loop ends (Line 149, after the last fold is processed)
#     KappaOPtimizer = minimize(evaluate_predictions,
#                               x0=[0.5, 1.5, 2.5], args=(y, oof_non_rounded), 
#                               method='Nelder-Mead')
#     assert KappaOPtimizer.success, "Optimization did not converge."

#     oof_tuned = threshold_Rounder(oof_non_rounded, KappaOPtimizer.x)
#     tKappa = quadratic_weighted_kappa(y, oof_tuned)

#     print(f"----> || Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {tKappa:.3f}{Style.RESET_ALL}")
    
#     tpm = test_preds.mean(axis=1)
#     tpTuned = threshold_Rounder(tpm, KappaOPtimizer.x)

#     # Create final submission prediction
#     submission = pd.DataFrame({
#         'id': sample['id'],
#         'sii': test_preds.mean(axis=1).round(0).astype(int)
#     })

#     return submission


# # LGBM model parameters
# Params = {
#     'learning_rate': 0.046,
#     'max_depth': 12,
#     'num_leaves': 478,
#     'min_data_in_leaf': 13,
#     'feature_fraction': 0.893,
#     'bagging_fraction': 0.784,
#     'bagging_freq': 4,
#     'lambda_l1': 10,  # Increased from 6.59
#     'lambda_l2': 0.01  # Increased from 2.68e-06
# }

# # Instantiate the LGBMRegressor model
# Light = LGBMRegressor(**Params, random_state=42, verbose=-1, n_estimators=300)

# # Train the model and get predictions
# Submission = TrainML(Light, test)

# # Output submission
# Submission.to_csv('submission.csv', index=False)


In [4]:
# from lightgbm import LGBMRegressor
# from sklearn.model_selection import StratifiedKFold
# from sklearn.metrics import cohen_kappa_score
# import numpy as np
# import pandas as pd
# from tqdm import tqdm
# from sklearn.base import clone
# from scipy.optimize import minimize
# import os
# from concurrent.futures import ThreadPoolExecutor
# from colorama import Fore, Style
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.impute import KNNImputer
# from sklearn.pipeline import Pipeline

# # Load data
# train = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/train.csv')
# test = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/test.csv')
# sample = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/sample_submission.csv')

# # Function to process time series data
# def process_file(filename, dirname):
#     df = pd.read_parquet(os.path.join(dirname, filename, 'part-0.parquet'))
#     df.drop('step', axis=1, inplace=True)
#     return df.describe().values.reshape(-1), filename.split('=')[1]

# def load_time_series(dirname) -> pd.DataFrame:
#     ids = os.listdir(dirname)
    
#     with ThreadPoolExecutor() as executor:
#         results = list(tqdm(executor.map(lambda fname: process_file(fname, dirname), ids), total=len(ids)))
    
#     stats, indexes = zip(*results)
    
#     df = pd.DataFrame(stats, columns=[f"stat_{i}" for i in range(len(stats[0]))])
#     df['id'] = indexes
#     return df
        
# train_ts = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet")
# test_ts = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet")

# time_series_cols = train_ts.columns.tolist()
# time_series_cols.remove("id")

# train = pd.merge(train, train_ts, how="left", on='id')
# test = pd.merge(test, test_ts, how="left", on='id')

# train = train.drop('id', axis=1)
# test = test.drop('id', axis=1)   

# # Feature columns
# featuresCols = ['Basic_Demos-Enroll_Season', 'Basic_Demos-Age', 'Basic_Demos-Sex',
#                 'CGAS-Season', 'CGAS-CGAS_Score', 'Physical-Season', 'Physical-BMI',
#                 'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
#                 'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
#                 'Fitness_Endurance-Season', 'Fitness_Endurance-Max_Stage',
#                 'Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec',
#                 'FGC-Season', 'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND',
#                 'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone', 'FGC-FGC_PU',
#                 'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR',
#                 'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone', 'BIA-Season',
#                 'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
#                 'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
#                 'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
#                 'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM',
#                 'BIA-BIA_TBW', 'PAQ_A-Season', 'PAQ_A-PAQ_A_Total', 'PAQ_C-Season',
#                 'PAQ_C-PAQ_C_Total', 'SDS-Season', 'SDS-SDS_Total_Raw',
#                 'SDS-SDS_Total_T', 'PreInt_EduHx-Season',
#                 'PreInt_EduHx-computerinternet_hoursday', 'sii']

# featuresCols += time_series_cols

# train = train[featuresCols]
# train = train.dropna(subset='sii')

# cat_c = ['Basic_Demos-Enroll_Season', 'CGAS-Season', 'Physical-Season', 
#           'Fitness_Endurance-Season', 'FGC-Season', 'BIA-Season', 
#           'PAQ_A-Season', 'PAQ_C-Season', 'SDS-Season', 'PreInt_EduHx-Season']

# def update(df):
#     global cat_c
#     for c in cat_c: 
#         df[c] = df[c].fillna('Missing')
#         df[c] = df[c].astype('category')
#     return df
        
# train = update(train)
# test = update(test)

# def create_mapping(column, dataset):
#     unique_values = dataset[column].unique()
#     return {value: idx for idx, value in enumerate(unique_values)}

# for col in cat_c:
#     mapping = create_mapping(col, train)
#     mappingTe = create_mapping(col, test)
    
#     train[col] = train[col].replace(mapping).astype(int)
#     test[col] = test[col].replace(mappingTe).astype(int)

# def quadratic_weighted_kappa(y_true, y_pred):
#     return cohen_kappa_score(y_true, y_pred, weights='quadratic')
# def threshold_Rounder(oof_non_rounded, thresholds):
#     return np.where(oof_non_rounded < thresholds[0], 0,
#                     np.where(oof_non_rounded < thresholds[1], 1,
#                              np.where(oof_non_rounded < thresholds[2], 2, 3)))
# def evaluate_predictions(thresholds, y_true, oof_non_rounded):
#     rounded_p = threshold_Rounder(oof_non_rounded, thresholds)
#     return -quadratic_weighted_kappa(y_true, rounded_p)



# def create_pipeline(model_class):
#     return Pipeline([
#         ('imputer', KNNImputer(n_neighbors=5)),
#         ('model', model_class)
#     ])

# def TrainML(model_class, test_data):
#     X = train.drop(['sii'], axis=1)
#     y = train['sii']

#     SKF = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
#     train_S = []
#     test_S = []
    
#     oof_non_rounded = np.zeros(len(y), dtype=float) 
#     oof_rounded = np.zeros(len(y), dtype=int) 
#     test_preds = np.zeros((len(test_data), 5))

#     for fold, (train_idx, test_idx) in enumerate(tqdm(SKF.split(X, y), desc="Training Folds", total=5)):
#         X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
#         y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]

#         # Create a new pipeline for each fold
#         pipeline = create_pipeline(clone(model_class))
#         pipeline.fit(X_train, y_train)

#         y_train_pred = pipeline.predict(X_train)
#         y_val_pred = pipeline.predict(X_val)

#         oof_non_rounded[test_idx] = y_val_pred
#         y_val_pred_rounded = y_val_pred.round(0).astype(int)
#         oof_rounded[test_idx] = y_val_pred_rounded

#         train_kappa = quadratic_weighted_kappa(y_train, y_train_pred.round(0).astype(int))
#         val_kappa = quadratic_weighted_kappa(y_val, y_val_pred_rounded)

#         train_S.append(train_kappa)
#         test_S.append(val_kappa)
        
#         test_preds[:, fold] = pipeline.predict(test_data)
        
#         print(f"Fold {fold+1} - Train QWK: {train_kappa:.4f}, Validation QWK: {val_kappa:.4f}")
    
#     print(f"Mean Train QWK --> {np.mean(train_S):.4f}")
#     print(f"Mean Validation QWK ---> {np.mean(test_S):.4f}")

#     KappaOptimizer = minimize(evaluate_predictions,
#                             x0=[0.5, 1.5, 2.5], args=(y, oof_non_rounded), 
#                             method='Nelder-Mead')
#     assert KappaOptimizer.success, "Optimization did not converge."

#     oof_tuned = threshold_Rounder(oof_non_rounded, KappaOptimizer.x)
#     tKappa = quadratic_weighted_kappa(y, oof_tuned)

#     print(f"----> || Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {tKappa:.3f}{Style.RESET_ALL}")
    
#     tpm = test_preds.mean(axis=1)
#     tpTuned = threshold_Rounder(tpm, KappaOptimizer.x)

#     # Create final submission prediction
#     submission = pd.DataFrame({
#         'id': sample['id'],
#         'sii': tpTuned  # Use the tuned predictions instead of rounded means
#     })

#     return submission

# # Random Forest parameters
# RF_Params = {
#     'n_estimators': 100,
#     'max_depth': 12,
#     'min_samples_split': 5,
#     'min_samples_leaf': 2,
#     'random_state': 42
# }

# # Instantiate the RandomForestRegressor model
# RF = RandomForestRegressor(**RF_Params)

# # Train the model and get predictions
# Submission = TrainML(RF, test)

# # Output submission
# Submission.to_csv('submission.csv', index=False)

In [5]:
# from lightgbm import LGBMRegressor
# from sklearn.model_selection import StratifiedKFold
# from sklearn.metrics import cohen_kappa_score
# import numpy as np
# import pandas as pd
# from tqdm import tqdm
# from sklearn.base import clone
# from scipy.optimize import minimize
# import os
# from concurrent.futures import ThreadPoolExecutor
# from colorama import Fore, Style
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.impute import KNNImputer
# from sklearn.pipeline import Pipeline

# from sklearn.ensemble import VotingRegressor, RandomForestRegressor, GradientBoostingRegressor

# # Load data
# train = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/train.csv')
# test = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/test.csv')
# sample = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/sample_submission.csv')

# # Function to process time series data
# def process_file(filename, dirname):
#     df = pd.read_parquet(os.path.join(dirname, filename, 'part-0.parquet'))
#     df.drop('step', axis=1, inplace=True)
#     return df.describe().values.reshape(-1), filename.split('=')[1]

# def load_time_series(dirname) -> pd.DataFrame:
#     ids = os.listdir(dirname)
    
#     with ThreadPoolExecutor() as executor:
#         results = list(tqdm(executor.map(lambda fname: process_file(fname, dirname), ids), total=len(ids)))
    
#     stats, indexes = zip(*results)
    
#     df = pd.DataFrame(stats, columns=[f"stat_{i}" for i in range(len(stats[0]))])
#     df['id'] = indexes
#     return df
        
# train_ts = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet")
# test_ts = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet")

# time_series_cols = train_ts.columns.tolist()
# time_series_cols.remove("id")

# train = pd.merge(train, train_ts, how="left", on='id')
# test = pd.merge(test, test_ts, how="left", on='id')

# train = train.drop('id', axis=1)
# test = test.drop('id', axis=1)   

# # Feature columns
# featuresCols = ['Basic_Demos-Enroll_Season', 'Basic_Demos-Age', 'Basic_Demos-Sex',
#                 'CGAS-Season', 'CGAS-CGAS_Score', 'Physical-Season', 'Physical-BMI',
#                 'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
#                 'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
#                 'Fitness_Endurance-Season', 'Fitness_Endurance-Max_Stage',
#                 'Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec',
#                 'FGC-Season', 'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND',
#                 'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone', 'FGC-FGC_PU',
#                 'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR',
#                 'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone', 'BIA-Season',
#                 'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
#                 'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
#                 'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
#                 'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM',
#                 'BIA-BIA_TBW', 'PAQ_A-Season', 'PAQ_A-PAQ_A_Total', 'PAQ_C-Season',
#                 'PAQ_C-PAQ_C_Total', 'SDS-Season', 'SDS-SDS_Total_Raw',
#                 'SDS-SDS_Total_T', 'PreInt_EduHx-Season',
#                 'PreInt_EduHx-computerinternet_hoursday', 'sii']

# featuresCols += time_series_cols

# train = train[featuresCols]
# train = train.dropna(subset='sii')

# cat_c = ['Basic_Demos-Enroll_Season', 'CGAS-Season', 'Physical-Season', 
#           'Fitness_Endurance-Season', 'FGC-Season', 'BIA-Season', 
#           'PAQ_A-Season', 'PAQ_C-Season', 'SDS-Season', 'PreInt_EduHx-Season']

# def update(df):
#     global cat_c
#     for c in cat_c: 
#         df[c] = df[c].fillna('Missing')
#         df[c] = df[c].astype('category')
#     return df
        
# train = update(train)
# test = update(test)

# def create_mapping(column, dataset):
#     unique_values = dataset[column].unique()
#     return {value: idx for idx, value in enumerate(unique_values)}

# for col in cat_c:
#     mapping = create_mapping(col, train)
#     mappingTe = create_mapping(col, test)
    
#     train[col] = train[col].replace(mapping).astype(int)
#     test[col] = test[col].replace(mappingTe).astype(int)

# def quadratic_weighted_kappa(y_true, y_pred):
#     return cohen_kappa_score(y_true, y_pred, weights='quadratic')
# def threshold_Rounder(oof_non_rounded, thresholds):
#     return np.where(oof_non_rounded < thresholds[0], 0,
#                     np.where(oof_non_rounded < thresholds[1], 1,
#                              np.where(oof_non_rounded < thresholds[2], 2, 3)))
# def evaluate_predictions(thresholds, y_true, oof_non_rounded):
#     rounded_p = threshold_Rounder(oof_non_rounded, thresholds)
#     return -quadratic_weighted_kappa(y_true, rounded_p)



# def create_pipeline(model_class):
#     return Pipeline([
#         ('imputer', KNNImputer(n_neighbors=5)),
#         ('model', model_class)
#     ])

# def TrainML(model_class, test_data):
#     X = train.drop(['sii'], axis=1)
#     y = train['sii']

#     SKF = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
#     train_S = []
#     test_S = []
    
#     oof_non_rounded = np.zeros(len(y), dtype=float) 
#     oof_rounded = np.zeros(len(y), dtype=int) 
#     test_preds = np.zeros((len(test_data), 5))

#     for fold, (train_idx, test_idx) in enumerate(tqdm(SKF.split(X, y), desc="Training Folds", total=5)):
#         X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
#         y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]

#         # Create a new pipeline for each fold
#         pipeline = create_pipeline(clone(model_class))
#         pipeline.fit(X_train, y_train)

#         y_train_pred = pipeline.predict(X_train)
#         y_val_pred = pipeline.predict(X_val)

#         oof_non_rounded[test_idx] = y_val_pred
#         y_val_pred_rounded = y_val_pred.round(0).astype(int)
#         oof_rounded[test_idx] = y_val_pred_rounded

#         train_kappa = quadratic_weighted_kappa(y_train, y_train_pred.round(0).astype(int))
#         val_kappa = quadratic_weighted_kappa(y_val, y_val_pred_rounded)

#         train_S.append(train_kappa)
#         test_S.append(val_kappa)
        
#         test_preds[:, fold] = pipeline.predict(test_data)
        
#         print(f"Fold {fold+1} - Train QWK: {train_kappa:.4f}, Validation QWK: {val_kappa:.4f}")
    
#     print(f"Mean Train QWK --> {np.mean(train_S):.4f}")
#     print(f"Mean Validation QWK ---> {np.mean(test_S):.4f}")

#     KappaOptimizer = minimize(evaluate_predictions,
#                             x0=[0.5, 1.5, 2.5], args=(y, oof_non_rounded), 
#                             method='Nelder-Mead')
#     assert KappaOptimizer.success, "Optimization did not converge."

#     oof_tuned = threshold_Rounder(oof_non_rounded, KappaOptimizer.x)
#     tKappa = quadratic_weighted_kappa(y, oof_tuned)

#     print(f"----> || Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {tKappa:.3f}{Style.RESET_ALL}")
    
#     tpm = test_preds.mean(axis=1)
#     tpTuned = threshold_Rounder(tpm, KappaOptimizer.x)

#     # Create final submission prediction
#     submission = pd.DataFrame({
#         'id': sample['id'],
#         'sii': tpTuned  # Use the tuned predictions instead of rounded means
#     })

#     return submission

# GB_Params = {
#     "n_estimators": 295,
#     "learning_rate": 0.012,
#     "max_depth": 3,
#     "min_samples_split": 18,
#     "min_samples_leaf": 5,
#     "subsample": 0.6,
#     "max_features": None,
#     "random_state": 42,
# }

# # Instantiate the RandomForestRegressor model
# GB = GradientBoostingRegressor(**GB_Params)

# # Train the model and get predictions
# Submission = TrainML(GB, test)

# # Output submission
# Submission.to_csv('submission.csv', index=False)

In [6]:
from sklearn.tree import DecisionTreeRegressor# from lightgbm import LGBMRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import cohen_kappa_score
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.base import clone
from scipy.optimize import minimize
import os
from concurrent.futures import ThreadPoolExecutor
from colorama import Fore, Style
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import KNNImputer
from sklearn.pipeline import Pipeline

from sklearn.ensemble import VotingRegressor, RandomForestRegressor, GradientBoostingRegressor


# Load data
train = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/train.csv')
test = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/test.csv')
sample = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/sample_submission.csv')

# Function to process time series data
def process_file(filename, dirname):
    df = pd.read_parquet(os.path.join(dirname, filename, 'part-0.parquet'))
    df.drop('step', axis=1, inplace=True)
    return df.describe().values.reshape(-1), filename.split('=')[1]

def load_time_series(dirname) -> pd.DataFrame:
    ids = os.listdir(dirname)
    
    with ThreadPoolExecutor() as executor:
        results = list(tqdm(executor.map(lambda fname: process_file(fname, dirname), ids), total=len(ids)))
    
    stats, indexes = zip(*results)
    
    df = pd.DataFrame(stats, columns=[f"stat_{i}" for i in range(len(stats[0]))])
    df['id'] = indexes
    return df
        
train_ts = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet")
test_ts = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet")

time_series_cols = train_ts.columns.tolist()
time_series_cols.remove("id")

train = pd.merge(train, train_ts, how="left", on='id')
test = pd.merge(test, test_ts, how="left", on='id')

train = train.drop('id', axis=1)
test = test.drop('id', axis=1)   

# Feature columns
featuresCols = ['Basic_Demos-Enroll_Season', 'Basic_Demos-Age', 'Basic_Demos-Sex',
                'CGAS-Season', 'CGAS-CGAS_Score', 'Physical-Season', 'Physical-BMI',
                'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
                'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
                'Fitness_Endurance-Season', 'Fitness_Endurance-Max_Stage',
                'Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec',
                'FGC-Season', 'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND',
                'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone', 'FGC-FGC_PU',
                'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR',
                'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone', 'BIA-Season',
                'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
                'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
                'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
                'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM',
                'BIA-BIA_TBW', 'PAQ_A-Season', 'PAQ_A-PAQ_A_Total', 'PAQ_C-Season',
                'PAQ_C-PAQ_C_Total', 'SDS-Season', 'SDS-SDS_Total_Raw',
                'SDS-SDS_Total_T', 'PreInt_EduHx-Season',
                'PreInt_EduHx-computerinternet_hoursday', 'sii']

featuresCols += time_series_cols

train = train[featuresCols]
train = train.dropna(subset='sii')

cat_c = ['Basic_Demos-Enroll_Season', 'CGAS-Season', 'Physical-Season', 
          'Fitness_Endurance-Season', 'FGC-Season', 'BIA-Season', 
          'PAQ_A-Season', 'PAQ_C-Season', 'SDS-Season', 'PreInt_EduHx-Season']

def update(df):
    global cat_c
    for c in cat_c: 
        df[c] = df[c].fillna('Missing')
        df[c] = df[c].astype('category')
    return df
        
train = update(train)
test = update(test)

def create_mapping(column, dataset):
    unique_values = dataset[column].unique()
    return {value: idx for idx, value in enumerate(unique_values)}

for col in cat_c:
    mapping = create_mapping(col, train)
    mappingTe = create_mapping(col, test)
    
    train[col] = train[col].replace(mapping).astype(int)
    test[col] = test[col].replace(mappingTe).astype(int)

def quadratic_weighted_kappa(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')
def threshold_Rounder(oof_non_rounded, thresholds):
    return np.where(oof_non_rounded < thresholds[0], 0,
                    np.where(oof_non_rounded < thresholds[1], 1,
                             np.where(oof_non_rounded < thresholds[2], 2, 3)))
def evaluate_predictions(thresholds, y_true, oof_non_rounded):
    rounded_p = threshold_Rounder(oof_non_rounded, thresholds)
    return -quadratic_weighted_kappa(y_true, rounded_p)

def create_pipeline(model_class):
    return Pipeline([
        ('imputer', KNNImputer(n_neighbors=5)),
        ('model', model_class)
    ])

def TrainML(model_class, test_data):
    X = train.drop(['sii'], axis=1)
    y = train['sii']

    SKF = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    train_S = []
    test_S = []
    
    oof_non_rounded = np.zeros(len(y), dtype=float) 
    oof_rounded = np.zeros(len(y), dtype=int) 
    test_preds = np.zeros((len(test_data), 5))

    for fold, (train_idx, test_idx) in enumerate(tqdm(SKF.split(X, y), desc="Training Folds", total=5)):
        X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]

        # Create a new pipeline for each fold
        pipeline = create_pipeline(clone(model_class))
        pipeline.fit(X_train, y_train)

        y_train_pred = pipeline.predict(X_train)
        y_val_pred = pipeline.predict(X_val)

        oof_non_rounded[test_idx] = y_val_pred
        y_val_pred_rounded = y_val_pred.round(0).astype(int)
        oof_rounded[test_idx] = y_val_pred_rounded

        train_kappa = quadratic_weighted_kappa(y_train, y_train_pred.round(0).astype(int))
        val_kappa = quadratic_weighted_kappa(y_val, y_val_pred_rounded)

        train_S.append(train_kappa)
        test_S.append(val_kappa)
        
        test_preds[:, fold] = pipeline.predict(test_data)
        
        print(f"Fold {fold+1} - Train QWK: {train_kappa:.4f}, Validation QWK: {val_kappa:.4f}")
    
    print(f"Mean Train QWK --> {np.mean(train_S):.4f}")
    print(f"Mean Validation QWK ---> {np.mean(test_S):.4f}")

    KappaOptimizer = minimize(evaluate_predictions,
                            x0=[0.5, 1.5, 2.5], args=(y, oof_non_rounded), 
                            method='Nelder-Mead')
    assert KappaOptimizer.success, "Optimization did not converge."

    oof_tuned = threshold_Rounder(oof_non_rounded, KappaOptimizer.x)
    tKappa = quadratic_weighted_kappa(y, oof_tuned)

    print(f"----> || Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {tKappa:.3f}{Style.RESET_ALL}")
    
    tpm = test_preds.mean(axis=1)
    tpTuned = threshold_Rounder(tpm, KappaOptimizer.x)

    # Create final submission prediction
    submission = pd.DataFrame({
        'id': sample['id'],
        'sii': tpTuned  # Use the tuned predictions instead of rounded means
    })

    return submission

DT_Params = {
    "max_depth": 5,
    "min_samples_split": 10,
    "min_samples_leaf": 4,
    "random_state": 42
}

# Instantiate the DecisionTreeRegressor model
DT = DecisionTreeRegressor(**DT_Params)

# Train the model and get predictions
Submission = TrainML(DT, test)

# Output submission
Submission.to_csv('submission.csv', index=False)


Training Folds:  20%|██        | 1/5 [00:07<00:28,  7.22s/it]

Fold 1 - Train QWK: 0.4467, Validation QWK: 0.3130


Training Folds:  40%|████      | 2/5 [00:14<00:21,  7.20s/it]

Fold 2 - Train QWK: 0.4168, Validation QWK: 0.3585


Training Folds:  60%|██████    | 3/5 [00:21<00:14,  7.28s/it]

Fold 3 - Train QWK: 0.4722, Validation QWK: 0.3068


Training Folds:  80%|████████  | 4/5 [00:29<00:07,  7.42s/it]

Fold 4 - Train QWK: 0.4810, Validation QWK: 0.2992


Training Folds: 100%|██████████| 5/5 [00:37<00:00,  7.40s/it]

Fold 5 - Train QWK: 0.4763, Validation QWK: 0.2287
Mean Train QWK --> 0.4586
Mean Validation QWK ---> 0.3013
----> || Optimized QWK SCORE ::  0.308
